<a href="https://colab.research.google.com/github/Jio-Kim/ML-DL/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 농구선수의 게임 기록을 학습해서 포지션 예측 by SVM(서포트 벡터 머신)

### 최적의 SVM 파라미터 찾기
- 두개의 파라미터 사용
  
  - 비용(C) : 마진의 너비 조정 파라미터
  - 감마(gamma) : 결정 경계선의 곡률 조정하는 파라미터

- 사이킷런의 그리드 서치(gridsearch)를 사용하면 최적의 비용과 감마 알아낼 수 있음
- 그리드 서치는 개발자가 부여한 비용과 감마 후보들을 모두 조합해 최적의 비용과 감마조합 찾아냄

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import numpy as np